In [1]:
!pip install -q anthropic openai pandas numpy tqdm datasets scikit-learn
!pip install google-generativeai together -q
!pip install cohere tenacity -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 70.3 MB/s eta 0:00:00


# -*- coding: utf-8 -*-
"""phases1_parallel.ipynb - FAST Parallel Processing Version

Performance improvements:
1. Async/concurrent API calls (10-50x faster)
2. Batch processing at multiple temperatures simultaneously
3. Thread pool for parallel execution
4. Smart rate limiting and retry logic
5. Resume capability maintained

Estimated speedup: 4 days → 4-12 hours depending on API limits
"""

In [2]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import time
from google.colab import drive
from datetime import datetime
import anthropic
import openai
from openai import OpenAI
import google.generativeai as genai
import cohere
from together import Together
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tenacity import retry, wait_exponential, stop_after_attempt
from collections import Counter
import asyncio
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

In [3]:
drive.mount('/content/drive')


PROJECT_ROOT = "/content/drive/MyDrive/PhD/Courses/year_2/text_analytics/POLAR_SemEval2026"
DATA_DIR = f"{PROJECT_ROOT}/data"
OUTPUT_DIR = f"{PROJECT_ROOT}/tier1_output"
PHASE1_OUTPUT = f"{OUTPUT_DIR}/phase1_multitemp"
os.makedirs(PHASE1_OUTPUT, exist_ok=True)


Mounted at /content/drive


In [ ]:
# API keys
os.environ["ANTHROPIC_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["DEEPSEEK_API_KEY"] = ""
os.environ["COHERE_API_KEY"] = ""
os.environ["TOGETHER_API_KEY"] = ""
# os.environ["GEMINI_API_KEY"] = xxxx

In [5]:
# Initialize clients
claude_client = anthropic.Anthropic()
openai_client = OpenAI()
deepseek_client = OpenAI(api_key=os.environ["DEEPSEEK_API_KEY"], base_url="https://api.deepseek.com")
# genai.configure(api_key=os.environ["GEMINI_API_KEY"])
gemini_client = gemini_client = genai.GenerativeModel('gemini-1.5-pro')
cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
together_client = Together()

In [6]:
# Languages
LANGUAGES = ['zho', 'tur', 'eng', 'arb', 'hin']
LANGUAGE_NAMES = {'zho': 'Chinese', 'tur': 'Turkish', 'eng': 'English', 'arb': 'Arabic', 'hin': 'Hindi'}

# Temperatures to test
TEMPERATURES = [0.0, 0.3, 0.7]

# PARALLEL PROCESSING SETTINGS
MAX_WORKERS = 20  # Number of concurrent API calls (adjust based on API limits)
BATCH_SIZE = 100  # Process this many samples before checkpoint
RATE_LIMIT_DELAY = 0.1  # Seconds between requests (adjust for API rate limits)



# System prompt
SYSTEM_PROMPT = """You are an expert at detecting online polarization.

POLARIZATION is content that creates sharp division into opposing groups with these characteristics:

POLARIZED content has:
- Us vs. Them framing: "our people" vs "those people", in-group vs out-group language
- Hostility or contempt toward a group (political, religious, racial, gender, etc.)
- Moral condemnation: portraying one side as evil/corrupt/dangerous
- Stereotyping or vilification of entire groups
- Dehumanizing language
- Zero-sum framing: if they win, we lose

NOT polarized:
- Factual reporting or neutral information
- Policy disagreement without hostility
- Criticism of specific actions/individuals without group-level attacks
- Personal opinions without divisive us-vs-them framing

Respond ONLY with JSON:
{
  "is_polarized": true/false,
  "confidence": 0.0-1.0,
  "reasoning": "brief explanation"
}"""


In [31]:
# Thread-safe rate limiter
class RateLimiter:
    def __init__(self, delay=0.1):
        self.delay = delay
        self.lock = threading.Lock()
        self.last_call = {}

    def wait(self, key='default'):
        with self.lock:
            now = time.time()
            if key in self.last_call:
                elapsed = now - self.last_call[key]
                if elapsed < self.delay:
                    time.sleep(self.delay - elapsed)
            self.last_call[key] = time.time()

rate_limiter = RateLimiter(RATE_LIMIT_DELAY)



@retry(wait=wait_exponential(multiplier=1, min=2, max=30), stop=stop_after_attempt(3))
def call_model_at_temp(text, language_name, model_name, temperature):
    """Call model at specific temperature with retry logic"""
    # Rate limiting
    rate_limiter.wait(model_name)

    try:
        result = None  # Initialize to avoid UnboundLocalError

        # Normalize model name for comparison (case-insensitive)
        model_name_lower = model_name.lower()

        if model_name_lower == 'deepseek':
            api_response = deepseek_client.chat.completions.create(
                model="deepseek-chat",
                temperature=temperature,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Language: {language_name}\n\nText: {text}\n\nAnalyze:"}
                ],
                timeout=30.0
            )
            response_text = api_response.choices[0].message.content.strip()

            # Remove markdown code blocks if present
            if response_text.startswith('```'):
                lines = response_text.split('\n')
                # Remove first line (```json or ```)
                lines = lines[1:]
                # Remove last line if it's ```
                if lines and lines[-1].strip() == '```':
                    lines = lines[:-1]
                response_text = '\n'.join(lines).strip()

            result = json.loads(response_text)

        elif model_name_lower == 'gpt-4o-mini':
            api_response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                temperature=temperature,
                response_format={"type": "json_object"},
                timeout=30.0,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Language: {language_name}\n\nText: {text}\n\nAnalyze:"}
                ]
            )
            response_text = api_response.choices[0].message.content.strip()
            result = json.loads(response_text)

        elif model_name_lower == 'qwen2.5':
            api_response = together_client.chat.completions.create(
                model="Qwen/Qwen2.5-72B-Instruct-Turbo",
                temperature=temperature,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": f"Language: {language_name}\n\nText: {text}\n\nAnalyze:"}
                ],
                timeout=30.0
            )
            result = json.loads(api_response.choices[0].message.content)

        else:
            raise ValueError(f"Unknown model: {model_name}")

        # Validate that we got a result
        if result is None:
            raise ValueError(f"No result generated for model: {model_name}")

        # Extract prediction
        predicted_val = result.get('is_polarized', None)
        if isinstance(predicted_val, bool):
            predicted_val = 1 if predicted_val else 0

        return {
            'prediction': predicted_val,
            'confidence': result.get('confidence', None),
            'error': None
        }

    except json.JSONDecodeError as e:
        return {
            'prediction': None,
            'confidence': None,
            'error': f'JSON decode error: {str(e)}'
        }
    except Exception as e:
        return {
            'prediction': None,
            'confidence': None,
            'error': str(e)
        }

In [18]:
def calculate_variance_confidence(predictions):
    """Calculate confidence from prediction variance"""
    valid_preds = [p for p in predictions if p is not None]
    if not valid_preds:
        return None, 0.0, True

    counts = Counter(valid_preds)
    majority_pred = counts.most_common(1)[0][0]
    agreement = counts[majority_pred] / len(valid_preds)
    variance_conf = agreement
    is_hard = agreement < 0.67

    return majority_pred, variance_conf, is_hard

def process_sample_temperature(row, model_name, temperature):
    """Process ONE sample at ONE temperature - for parallel execution"""
    try:
        pred = call_model_at_temp(row['text'], row['language_name'], model_name, temperature)
        return {
            'row_id': row['id'],
            'model': model_name,
            'temp': temperature,
            'prediction': pred['prediction'],
            'confidence': pred['confidence'],
            'error': pred['error']
        }
    except Exception as e:
        return {
            'row_id': row['id'],
            'model': model_name,
            'temp': temperature,
            'prediction': None,
            'confidence': None,
            'error': str(e)
        }

def process_batch_parallel(df_batch, model_name, max_workers=MAX_WORKERS):
    """
    Process a batch of samples in parallel across all temperatures
    This is the KEY speedup - all temperatures for all samples run concurrently
    """
    tasks = []

    # Create tasks for all samples and all temperatures
    for idx, row in df_batch.iterrows():
        for temp in TEMPERATURES:
            tasks.append((row, model_name, temp))

    results = []

    # Execute all tasks in parallel
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_task = {
            executor.submit(process_sample_temperature, row, model, temp): (row['id'], model, temp)
            for row, model, temp in tasks
        }

        # Collect results as they complete
        for future in tqdm(as_completed(future_to_task),
                          total=len(future_to_task),
                          desc=f"Processing batch ({len(df_batch)} samples × {len(TEMPERATURES)} temps)",
                          leave=False):
            try:
                result = future.result()
                results.append(result)
            except Exception as e:
                row_id, model, temp = future_to_task[future]
                results.append({
                    'row_id': row_id,
                    'model': model,
                    'temp': temp,
                    'prediction': None,
                    'confidence': None,
                    'error': str(e)
                })

    return results


In [32]:
def aggregate_results(df_batch, parallel_results, model_name):
    """Convert parallel results back to the expected format"""
    aggregated = []

    for idx, row in df_batch.iterrows():
        result = {
            'id': row['id'],
            'text': row['text'],
            'language': row['language'],
            'language_name': row['language_name'],
            'true_label': int(row['polarization'])
        }

        # Get all predictions for this sample
        sample_results = [r for r in parallel_results if r['row_id'] == row['id']]

        model_preds = []
        model_confs = []

        # Organize by temperature
        for temp in TEMPERATURES:
            temp_result = next((r for r in sample_results if r['temp'] == temp), None)

            if temp_result:
                result[f'{model_name}_t{temp}_pred'] = temp_result['prediction']
                result[f'{model_name}_t{temp}_conf'] = temp_result['confidence']
                result[f'{model_name}_t{temp}_error'] = temp_result['error']

                if temp_result['prediction'] is not None:
                    model_preds.append(temp_result['prediction'])
                if temp_result['confidence'] is not None:
                    model_confs.append(temp_result['confidence'])

        # Calculate aggregates
        maj_pred, maj_conf, is_uncertain = calculate_variance_confidence(model_preds)
        result[f'{model_name}_majority'] = maj_pred
        result[f'{model_name}_var_confidence'] = maj_conf
        result[f'{model_name}_uncertain'] = is_uncertain
        result[f'{model_name}_avg_confidence'] = np.mean(model_confs) if model_confs else None

        aggregated.append(result)

    return aggregated

In [33]:
def load_full_train_data():
    """Load all training samples"""
    all_data = []
    for lang in LANGUAGES:
        df = pd.read_csv(f"{DATA_DIR}/subtask1/train/{lang}.csv")
        df['language'] = lang
        df['language_name'] = LANGUAGE_NAMES[lang]
        all_data.append(df)
    return pd.concat(all_data, ignore_index=True)

def load_existing_results(model_name):
    """Load existing results from checkpoint or final file"""
    output_path = f"{PHASE1_OUTPUT}/{model_name}_predictions.csv"

    if os.path.exists(output_path):
        print(f"Found existing results file: {output_path}")
        df = pd.read_csv(output_path)
        return df, len(df)

    checkpoint_files = [f for f in os.listdir(PHASE1_OUTPUT)
                       if f.startswith(f"{model_name}_checkpoint_") and f.endswith('.csv')]

    if checkpoint_files:
        checkpoint_nums = [int(f.split('_')[-1].replace('.csv', '')) for f in checkpoint_files]
        latest_checkpoint_num = max(checkpoint_nums)
        latest_checkpoint = f"{PHASE1_OUTPUT}/{model_name}_checkpoint_{latest_checkpoint_num}.csv"

        print(f"Found checkpoint: {latest_checkpoint}")
        df = pd.read_csv(latest_checkpoint)
        return df, len(df)

    return None, 0

In [34]:
def run_model_parallel(model_name,
                      batch_size=BATCH_SIZE,
                      max_workers=MAX_WORKERS,
                      force_restart=False):
    """
    Run predictions with PARALLEL processing - MUCH FASTER!

    Args:
        model_name: 'DeepSeek', 'GPT-4o', or 'Qwen2.5'
        batch_size: Process this many samples before checkpoint (larger = faster but less frequent saves)
        max_workers: Number of concurrent API calls (adjust based on rate limits)
        force_restart: If True, ignore existing results and start fresh

    Speed comparison:
        Sequential: ~4 days for 5000 samples (assuming 1 sec per call × 3 temps = 3 sec each)
        Parallel (20 workers): ~4-6 hours (20x speedup)
        Parallel (50 workers): ~2-3 hours (40x speedup, if API allows)
    """
    print(f"\n{'='*60}")
    print(f"PARALLEL Processing: {model_name}")
    print(f"Settings: {max_workers} workers, batch size {batch_size}")
    print(f"{'='*60}")

    # Load full data
    df = load_full_train_data()
    total_samples = len(df)

    # Check for existing results
    existing_results = None
    resume_from_idx = 0

    if not force_restart:
        existing_results, processed_count = load_existing_results(model_name)
        if existing_results is not None and processed_count > 0:
            resume_from_idx = processed_count
            print(f"Resuming from index {resume_from_idx} ({processed_count} samples already processed)")
            print(f"Progress: {processed_count}/{total_samples} ({processed_count/total_samples*100:.1f}%)")

            user_input = input(f"\nContinue from index {resume_from_idx}? (y/n): ").strip().lower()
            if user_input != 'y':
                print("Starting fresh...")
                existing_results = None
                resume_from_idx = 0

    # Get remaining data
    df_to_process = df.iloc[resume_from_idx:].copy().reset_index(drop=True)

    # Initialize results
    if existing_results is not None:
        all_results = existing_results.to_dict('records')
    else:
        all_results = []

    # Process in batches
    num_batches = (len(df_to_process) + batch_size - 1) // batch_size

    print(f"\nProcessing {len(df_to_process)} samples in {num_batches} batches")
    print(f"Estimated time: {len(df_to_process) * len(TEMPERATURES) / max_workers / 60:.1f} minutes")
    print(f"(Compared to sequential: {len(df_to_process) * len(TEMPERATURES) / 60:.1f} minutes)\n")

    start_time = time.time()

    for batch_idx in range(0, len(df_to_process), batch_size):
        batch_num = batch_idx // batch_size + 1
        batch_end = min(batch_idx + batch_size, len(df_to_process))
        df_batch = df_to_process.iloc[batch_idx:batch_end]

        print(f"\n[Batch {batch_num}/{num_batches}] Processing samples {resume_from_idx + batch_idx} to {resume_from_idx + batch_end}")

        # PARALLEL PROCESSING - This is where the magic happens!
        parallel_results = process_batch_parallel(df_batch, model_name, max_workers)

        # Aggregate results
        batch_results = aggregate_results(df_batch, parallel_results, model_name)
        all_results.extend(batch_results)

        # Save checkpoint
        checkpoint_df = pd.DataFrame(all_results)
        checkpoint_path = f"{PHASE1_OUTPUT}/{model_name}_checkpoint_{len(all_results)}.csv"
        checkpoint_df.to_csv(checkpoint_path, index=False)

        elapsed = time.time() - start_time
        samples_done = len(all_results) - (0 if existing_results is None else len(existing_results))
        rate = samples_done / elapsed if elapsed > 0 else 0
        remaining = len(df_to_process) - (batch_end - batch_idx)
        eta_seconds = remaining / rate if rate > 0 else 0

        print(f"✓ Checkpoint saved: {len(all_results)}/{total_samples} samples")
        print(f"  Speed: {rate:.1f} samples/sec")
        print(f"  ETA: {eta_seconds/60:.1f} minutes")

    # Save final results
    results_df = pd.DataFrame(all_results)
    output_path = f"{PHASE1_OUTPUT}/{model_name}_predictions.csv"
    results_df.to_csv(output_path, index=False)

    # Clean up checkpoints
    checkpoint_files = [f for f in os.listdir(PHASE1_OUTPUT)
                       if f.startswith(f"{model_name}_checkpoint_") and f.endswith('.csv')]
    for checkpoint_file in checkpoint_files:
        try:
            os.remove(f"{PHASE1_OUTPUT}/{checkpoint_file}")
        except:
            pass

    # Calculate metrics (same as before)
    print(f"\n{'='*60}")
    print(f"{model_name} - Results Summary")
    print(f"{'='*60}")

    valid_mask = results_df[f'{model_name}_majority'].notna()
    valid_results = results_df[valid_mask]

    if len(valid_results) > 0:
        y_true = valid_results['true_label']
        y_pred = valid_results[f'{model_name}_majority']

        accuracy = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)

        total_time = time.time() - start_time

        print(f"Total samples: {len(results_df)}")
        print(f"Valid predictions: {len(valid_results)} ({len(valid_results)/len(results_df)*100:.1f}%)")
        print(f"Total time: {total_time/60:.1f} minutes ({total_time/3600:.2f} hours)")
        print(f"Average speed: {len(results_df)/total_time:.2f} samples/second")
        print(f"\nPerformance Metrics:")
        print(f"  Accuracy:  {accuracy:.4f}")
        print(f"  F1 Score:  {f1:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")

        # Per-temperature metrics
        print(f"\nPer-Temperature Performance:")
        for temp in TEMPERATURES:
            temp_valid = results_df[f'{model_name}_t{temp}_pred'].notna()
            if temp_valid.sum() > 0:
                temp_results = results_df[temp_valid]
                temp_acc = accuracy_score(temp_results['true_label'], temp_results[f'{model_name}_t{temp}_pred'])
                temp_f1 = f1_score(temp_results['true_label'], temp_results[f'{model_name}_t{temp}_pred'])
                print(f"  T={temp}: Acc={temp_acc:.4f}, F1={temp_f1:.4f}")

        # Confidence analysis
        avg_var_conf = valid_results[f'{model_name}_var_confidence'].mean()
        avg_model_conf = valid_results[f'{model_name}_avg_confidence'].mean()
        uncertain_pct = valid_results[f'{model_name}_uncertain'].sum() / len(valid_results) * 100

        print(f"\nConfidence Analysis:")
        print(f"  Avg variance confidence: {avg_var_conf:.4f}")
        print(f"  Avg model confidence: {avg_model_conf:.4f}")
        print(f"  Uncertain samples: {uncertain_pct:.1f}%")

        # Per-language breakdown
        print(f"\nPer-Language Performance:")
        for lang in sorted(valid_results['language'].unique()):
            lang_data = valid_results[valid_results['language'] == lang]
            if len(lang_data) > 0:
                lang_acc = accuracy_score(lang_data['true_label'], lang_data[f'{model_name}_majority'])
                lang_f1 = f1_score(lang_data['true_label'], lang_data[f'{model_name}_majority'])
                print(f"  {lang}: N={len(lang_data)}, Acc={lang_acc:.4f}, F1={lang_f1:.4f}")

    print(f"\nResults saved to: {output_path}")
    print(f"{'='*60}\n")

    return results_df

In [35]:
run_model_parallel('gpt-4o-mini')



PARALLEL Processing: gpt-4o-mini
Settings: 20 workers, batch size 100

Processing 15444 samples in 155 batches
Estimated time: 38.6 minutes
(Compared to sequential: 772.2 minutes)


[Batch 1/155] Processing samples 0 to 100


Processing batch (100 samples × 3 temps):   0%|          | 0/300 [00:00<?, ?it/s]

✓ Checkpoint saved: 100/15444 samples
  Speed: 2.9 samples/sec
  ETA: 89.6 minutes

[Batch 2/155] Processing samples 100 to 200


Processing batch (100 samples × 3 temps):   0%|          | 0/300 [00:00<?, ?it/s]

✓ Checkpoint saved: 200/15444 samples
  Speed: 2.7 samples/sec
  ETA: 94.2 minutes

[Batch 3/155] Processing samples 200 to 300


...

Processing batch (100 samples × 3 temps):   0%|          | 0/300 [00:00<?, ?it/s]

✓ Checkpoint saved: 15400/15444 samples
  Speed: 0.5 samples/sec
  ETA: 532.4 minutes

[Batch 155/155] Processing samples 15400 to 15444


Processing batch (44 samples × 3 temps):   0%|          | 0/132 [00:00<?, ?it/s]

✓ Checkpoint saved: 15444/15444 samples
  Speed: 0.5 samples/sec
  ETA: 533.3 minutes

gpt-4o-mini - Results Summary
Total samples: 15444
Valid predictions: 6180 (40.0%)
Total time: 534.9 minutes (8.91 hours)
Average speed: 0.48 samples/second

Performance Metrics:
  Accuracy:  0.8388
  F1 Score:  0.8494
  Precision: 0.8783
  Recall:    0.8223

Per-Temperature Performance:
  T=0.0: Acc=0.8471, F1=0.8613
  T=0.3: Acc=0.8543, F1=0.8668
  T=0.7: Acc=0.8481, F1=0.8634

Confidence Analysis:
  Avg variance confidence: 0.9951
  Avg model confidence: 0.8686
  Uncertain samples: 1.4%

Per-Language Performance:
  arb: N=842, Acc=0.7850, F1=0.7763
  eng: N=646, Acc=0.8251, F1=0.7753
  hin: N=487, Acc=0.7721, F1=0.8498
  tur: N=638, Acc=0.7837, F1=0.7915
  zho: N=3567, Acc=0.8730, F1=0.8838

Results saved to: /content/drive/MyDrive/PhD/Courses/year_2/text_analytics/POLAR_SemEval2026/tier1_output/phase1_multitemp/gpt-4o-mini_predictions.csv



,id,text,language,language_name,true_label,gpt-4o-mini_t0.0_pred,gpt-4o-mini_t0.0_conf,gpt-4o-mini_t0.0_error,gpt-4o-mini_t0.3_pred,gpt-4o-mini_t0.3_conf,gpt-4o-mini_t0.3_error,gpt-4o-mini_t0.7_pred,gpt-4o-mini_t0.7_conf,gpt-4o-mini_t0.7_error,gpt-4o-mini_majority,gpt-4o-mini_var_confidence,gpt-4o-mini_uncertain,gpt-4o-mini_avg_confidence
0,zho_e59ce789a1d83d91cb7d362efaa3bf23,好久都没有见过那么干净的白人班级了，一个黑乐色都没有,zho,Chinese,1,1.0,0.90,None,1.0,0.90,None,1.0,0.90,None,1.0,1.0,False,0.900000
1,zho_ea3ef5567698fc40fa631baf7f697a16,楼主这讽刺太有力度把跪虫族和国际鬼子虫类及其它们后代虫混描写淋漓尽致,zho,Chinese,1,1.0,0.90,None,1.0,0.90,None,1.0,0.90,None,1.0,1.0,False,0.900000
2,zho_b73779673957f6dab5688c27a6747458,你这样说让那些标榜找个外国对象倍有面的人还怎么装b�,zho,Chinese,1,1.0,0.80,None,1.0,0.85,None,1.0,0.80,None,1.0,1.0,False,0.816667
3,zho_f7164a14baafbcb6ec387b0217a1c766,说明一下，我只关爱自己身边的女性，比如老婆女儿等，别的女人关我鸟事。v,zho,Chinese,1,1.0,0.85,None,1.0,0.85,None,1.0,0.85,None,1.0,1.0,False,0.850000
4,zho_1be506c572b805494787f548671a5bb7,基佬还说之前视频里的不是它，它都说照片是它爆出来的了。,zho,Chinese,1,1.0,0.80,None,1.0,0.80,None,1.0,0.85,None,1.0,1.0,False,0.816667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15439,hin_73da6c5d574ddb7b3c3078292cd828c6,शम॔ तो उसने बेच खाई है। अब कैसे आयेगी? कांग्रे...,hin,Hindi,1,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'Rate ...,NaN,NaN,Error code: 429 - {'error': {'message': 'Rate ...,NaN,0.0,True,NaN
15440,hin_49eda0302aecdb04adf1b78dc7382051,गुजरात उसी पार्टी को वोट देगा जिसमे देश प्रेम ...,hin,Hindi,1,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,0.0,True,NaN
15441,hin_87b67739af0a3c87846ffbff1f600617,पूरे देश की आबादी में भी 35 % हो चुके हैं पर त...,hin,Hindi,1,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,0.0,True,NaN
15442,hin_7d9fbb5dad9da13566b525482b2ef8cd,कब तक करोगी और मुस्लिम की तुष्टीकरण अब तो बंद ...,hin,Hindi,1,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,NaN,NaN,Error code: 429 - {'error': {'message': 'You e...,1.0,0.90,None,1.0,1.0,False,0.900000


✗ Quota exceeded - you've hit your limit
